In [1]:
# As always we start by importing the necessary libraries
# and loading the data we will be working with. (at this point I'm importing stuff just for the fun of it)
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utility.classification_utility import *

# we define a random state to make the results reproducible
RANDOM_STATE = 42
RUN_SLOW_STUFF = True

cyc = '../dataset/cyclists_cleaned.csv'
races = '../dataset/races_cleaned.csv'

full_df = get_merged_dataset(cyc, races)
full_df = define_target(full_df) 

In [4]:
#full_df = define_target(full_df)
full_df[['points', 'position', 'average_speed', 'stamina_index']]
print('points:', full_df['points'].isna().sum())
print('position:',full_df['position'].isna().sum())
print('average_speed:',full_df['average_speed'].isna().sum())
print('stamina_index:',full_df['stamina_index'].isna().sum())

points: 425399
position: 0
average_speed: 0
stamina_index: 137267


In [17]:
full_df.columns

Index(['_url_rac', 'name_rac', 'stage', 'stage_type', 'points', 'uci_points',
       'length', 'climb_total', 'profile', 'startlist_quality', 'date',
       'position', 'cyclist', 'cyclist_age_rac', 'is_tarmac', 'delta', 'time',
       'time_seconds', 'average_speed', 'steepness', 'season', 'is_staged',
       'race_country', 'norm_points', 'age_performance_index',
       'quality_adjusted_points', 'normalized_length', 'normalized_quality',
       'normalized_steepness', 'normalized_time', 'stamina_index', '_url_cyc',
       'name_cyc', 'birth_year', 'weight', 'height', 'nationality', 'bmi',
       'race_count', 'experience_level', 'total_points', 'victories_by_points',
       'avg_points_per_race', 'average_position', 'avg_speed_cyclist',
       'cyclist_age_cyc', 'mean_stamina_index'],
      dtype='object')

In [ ]:
import os
import sys
from random import shuffle
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

from xgboost import XGBClassifier # XGBoost classifier
from time import time # to compute time
from itertools import product #for grid search
from tqdm import tqdm # for progress bar
from sklearn.metrics import classification_report, f1_score # for evaluation
from utilsData import dataset_loader, load_data # for loading data
from imblearn.over_sampling import RandomOverSampler # for oversampling
f1_macro = lambda x, y: f1_score(x, y, average='macro') # for our evaluation

folderName = f'./Datasets/Cleaned_Dataset_{years_to_death}Y/chl_dataset_known.csv'

# TODO: complete gridsearch adding various sampling strategies
oversample = RandomOverSampler(sampling_strategy='minority', random_state=42)
tr_data, tr_out = oversample.fit_resample(tr_data, tr_out)

# Hyperparameters
N_estimators = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400]
Learning_rate = [0.001, 0.005, 0.01, 0.05, 0.1]
# Shuffle hyperparameters
hyperparameters = list(product(N_estimators, Learning_rate))
shuffle(hyperparameters)

# Grid search
best_score = 0
best_params = None
for n_estimators, learning_rate in tqdm(hyperparameters, total=len(hyperparameters)):
    xgb_model = XGBClassifier(
        n_estimators=n_estimators, 
        learning_rate=learning_rate, 
        n_jobs=-1, 
        random_state=42,
        objective='binary:logistic'
    )
    xgb_model.fit(tr_data, tr_out)
    val_pred = xgb_model.predict(val_data)
    score = f1_macro(val_out, val_pred)
    if score > best_score:
        best_score = score
        best_params = (n_estimators, learning_rate)

# Train the model with the best hyperparameters
n_estimators, learning_rate = best_params
xgb_model = XGBClassifier(
    n_estimators=n_estimators, 
    learning_rate=learning_rate, 
    n_jobs=-1, 
    random_state=42,
    objective='binary:logistic'
)
xgb_model.fit(tr_data, tr_out)

# Predict
val_pred = xgb_model.predict(val_data)
print(f'val f1-score: {f1_macro(val_out, val_pred)}')
print(classification_report(val_out, val_pred))
print(f'Best hyperparameters: {best_params}')
print(f'Best score: {best_score}')